In [ ]:
%%capture
%pip install transformers[torch]
%pip install accelerate -U
%pip install datasets

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline
from datasets import Dataset
from sklearn.metrics import precision_recall_fscore_support

# Обучение модели

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Hakaton/train_data.csv')

train['full'] = train.question_2 + train.question_4

pos_train = train[['full', 'object']]
pos_train.columns = ['text', 'labels']

ds_pos = Dataset.from_pandas(pos_train)

train.head(5)

,timestamp,question_1,question_2,question_3,question_4,question_5,is_relevant,object,is_positive,full
0,2024-04-01 08:30:00,Основы программирования,Интерактивный подход к изучению основ C# был о...,Нет.,Увеличить количество практических заданий для ...,Введение в разработку на .NET Framework.,1,0,1,Интерактивный подход к изучению основ C# был о...
1,2024-04-01 10:00:00,Основы программирования,Подробное объяснение логических операторов в J...,Нет.,Предоставить ещё больше практических задач пос...,Детальное изучение асинхронного программирован...,1,0,1,Подробное объяснение логических операторов в J...
2,2024-04-01 15:00:00,Основы программирования,Введение в программирование на Python было про...,Нет.,"Улучшить качество видеоматериалов, иногда было...",Библиотеки Python для анализа данных и машинно...,1,0,1,Введение в программирование на Python было про...
3,2024-04-01 17:00:00,Основы программирования,"Понятное изложение основных понятий, хорошая с...","Да, циклы for и while были объяснены слишком б...",Добавить видеоматериалы для самостоятельного и...,Функциональное программирование.,1,0,1,"Понятное изложение основных понятий, хорошая с..."
4,2024-04-02 09:00:00,Основы программирования,Кларность и структура материала по основам Jav...,Нет.,Ввести больше примеров на работу с массивами и...,Работа с асинхронным кодом в JavaScript.,1,0,1,Кларность и структура материала по основам Jav...


In [ ]:
initial = 'DeepPavlov/rubert-base-cased-conversational'

tokenizer = AutoTokenizer.from_pretrained(initial)

def tokenize(df):
  return tokenizer(df['text'], max_length=512, padding=True, truncation=True, return_tensors='pt')

model = AutoModelForSequenceClassification.from_pretrained(
    initial,
    num_labels=3,
    label2id={'webinar': 0, 'program': 1, 'tutor': 2},
    id2label={0: 'webinar', 1: 'program', 2: 'tutor'}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_args = TrainingArguments(output_dir='test',
                               num_train_epochs=3,
                               evaluation_strategy='epoch')

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, f1 = precision_recall_fscore_support(labels, preds, average='macro')[:3]
  print({'precision' : precision, 'recall' : recall, 'F1-score' : f1})
  return {'precision' : precision, 'recall' : recall, 'F1-score' : f1}

trainer = Trainer(model,
                  args=train_args,
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer,
                  train_dataset=ds_pos.map(tokenize, batched=True),
                  eval_dataset=ds_pos.map(tokenize, batched=True),
                  data_collator=data_collator)

Map:   0%|          | 0/103 [00:00<?, ? examples/s]

Map:   0%|          | 0/103 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1-score
1,No log,1.022117,0.352039,0.431579,0.354219
2,No log,0.701979,0.829607,0.758563,0.756389
3,No log,0.542309,0.918246,0.906266,0.909104


{'precision': 0.35203890759446316, 'recall': 0.43157894736842106, 'F1-score': 0.35421888053467}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'precision': 0.8296070460704609, 'recall': 0.7585630743525481, 'F1-score': 0.7563888579800452}
{'precision': 0.9182456140350878, 'recall': 0.9062656641604011, 'F1-score': 0.9091036817064214}


TrainOutput(global_step=39, training_loss=0.8955300159943409, metrics={'train_runtime': 11.129, 'train_samples_per_second': 27.765, 'train_steps_per_second': 3.504, 'total_flos': 15085340580870.0, 'train_loss': 0.8955300159943409, 'epoch': 3.0})

In [ ]:
trainer.save_model('/content/drive/MyDrive/Hakaton/Models_for_Kolbasa/object_model')

In [ ]:
pipe = pipeline("text-classification",
                 model='/content/drive/MyDrive/Hakaton/Models_for_Kolbasa')

In [ ]:
pipe(train.full.to_list())

[{'label': 'positive', 'score': 0.9746651649475098},
 {'label': 'positive', 'score': 0.9765839576721191},
 {'label': 'positive', 'score': 0.9741864800453186},
 {'label': 'positive', 'score': 0.9709163904190063},
 {'label': 'positive', 'score': 0.9787499904632568},
 {'label': 'positive', 'score': 0.9785037636756897},
 {'label': 'negative', 'score': 0.907088577747345},
 {'label': 'positive', 'score': 0.977192223072052},
 {'label': 'positive', 'score': 0.9778900742530823},
 {'label': 'positive', 'score': 0.9797890782356262},
 {'label': 'positive', 'score': 0.9764467477798462},
 {'label': 'positive', 'score': 0.9776480197906494},
 {'label': 'positive', 'score': 0.9771883487701416},
 {'label': 'negative', 'score': 0.9039027690887451},
 {'label': 'positive', 'score': 0.9730905890464783},
 {'label': 'positive', 'score': 0.9718046188354492},
 {'label': 'positive', 'score': 0.9729581475257874},
 {'label': 'positive', 'score': 0.9789209961891174},
 {'label': 'positive', 'score': 0.97511631250381